<a href="https://colab.research.google.com/github/Aashutosh-Basnet/Aashutosh-Basnet/blob/main/nn_iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


In [18]:
iris = load_iris()
X = iris.data
y = iris.target

In [19]:
encoder = OneHotEncoder()
y_encoded = encoder.fit_transform(y.reshape(-1, 1)).toarray()

In [20]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

In [22]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(120, 4)
(30, 4)
(120, 3)
(30, 3)


In [23]:
input_neurons = X_train.shape[1]
hidden_neurons = 8
output_neurons = y_train.shape[1]

In [24]:
np.random.seed(42)
W1 = np.random.rand(input_neurons, hidden_neurons) - 0.5
b1 = (np.random.rand(hidden_neurons) - 0.5).reshape(-1)
W2 = np.random.rand(hidden_neurons, output_neurons) - 0.5
b2 = (np.random.rand(output_neurons) - 0.5).reshape(-1)

In [25]:
def relu(x):
  return np.maximum(0, x)

def relu_derivative(x):
  return np.where(x > 0, 1, 0)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1).reshape(-1, 1))
    return exp_x / np.sum(exp_x, axis=1).reshape(-1, 1)

In [26]:
# Forward propagation
def forward_propagation(X):
    # Hidden layer
    Z1 = np.dot(X, W1) + b1
    A1 = relu(Z1)
    # Output layer
    Z2 = np.dot(A1, W2) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

# Backward propagation
def backward_propagation(X, y, Z1, A1, Z2, A2, learning_rate=0.01):
    global W1, b1, W2, b2

    # Output layer error
    dZ2 = A2 - y
    dW2 = np.dot(A1.T, dZ2) / X.shape[0]
    db2 = np.sum(dZ2, axis=0) / X.shape[0]

    # Hidden layer error
    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * relu_derivative(Z1)
    dW1 = np.dot(X.T, dZ1) / X.shape[0]
    db1 = np.sum(dZ1, axis=0) / X.shape[0]

    # Update weights and biases
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2

In [27]:
# Training parameters
epochs = 1000
learning_rate = 0.01

# Training loop
for epoch in range(epochs):
    # Forward propagation
    Z1, A1, Z2, A2 = forward_propagation(X_train)

    # Backward propagation
    backward_propagation(X_train, y_train, Z1, A1, Z2, A2, learning_rate)

    # Compute loss (cross-entropy)
    loss = -np.mean(np.sum(y_train * np.log(A2 + 1e-8), axis=1))

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

# Test the model
_, _, _, A2_test = forward_propagation(X_test)
y_pred = np.argmax(A2_test, axis=1)
y_true = np.argmax(y_test, axis=1)

# Compute accuracy
accuracy = np.mean(y_pred == y_true)
print(f"Test Accuracy: {accuracy:.2f}")


Epoch 0, Loss: 1.1374
Epoch 100, Loss: 0.7133
Epoch 200, Loss: 0.5508
Epoch 300, Loss: 0.4664
Epoch 400, Loss: 0.4164
Epoch 500, Loss: 0.3832
Epoch 600, Loss: 0.3587
Epoch 700, Loss: 0.3385
Epoch 800, Loss: 0.3209
Epoch 900, Loss: 0.3049
Test Accuracy: 0.97
